In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import importlib
import download_clean_data as dc
import ipca
import metrics
import kernel_regression as kr
import cross_validaton as cv
importlib.reload(dc) 
importlib.reload(ipca)
importlib.reload(metrics)
importlib.reload(kr)
importlib.reload(cv)

<module 'cross_validaton' from '/Users/matteoferrazzi/Documents/GitHub/ML-project-2/cross_validaton.py'>

In [3]:
folder_path = "/Users/matteoferrazzi/Documents/GitHub/ML-project-2/Data/monthly_data"

In [4]:
N = 100
date = 20150000
data, ret = dc.download_clean_data(folder_path, date, N)

In [5]:
x = []
for t in range(len(ret)-1):
    x.append(data[t].values.T@ret[t+1].values/len(ret[0]))

x_cov = np.sum([x[i].reshape((-1,1))@x[i].reshape((1,-1)) for i in range(len(x))], axis = 0)
eigval_x, eigvec_x = np.linalg.eig(x_cov)

idx = np.argsort(np.abs(eigval_x))[::-1]
sort_eigvec_x = eigvec_x[:,idx].real
k = 5
gamma_first = sort_eigvec_x[:,:k]

In [6]:
max_iter = 100
gamma, f_list = ipca.ipca(data, ret, gamma_first.copy(), max_iter)
W = np.eye(N)
W_list = [W]*(len(data)-1)
lambda1 = 0.1
lambda2 = 0.1
gamma_reg_w, f_list_reg_w = ipca.ipca_reg_w(data, ret, gamma_first.copy(), max_iter, lambda1, lambda2, W_list)

In [31]:
print(metrics.total_R_squared(ret, data, gamma, f_list),metrics.pred_R_squared(ret, data, gamma, f_list))

0.15335737000068583 -0.004556714482517377


In [32]:
print(metrics.total_R_squared(ret, data, gamma_reg_w, f_list_reg_w),metrics.pred_R_squared(ret, data, gamma_reg_w, f_list_reg_w))

0.15167256959390363 -0.0028241664513473097


In [14]:
y = ret
x = data
trsh = 0.8
lambda1_v = list([0.1])
lambda2_v = list([0.1])

cv.cross_val_IPCA_reg(y,x, trsh, lambda1_v, lambda2_v, gamma_first, max_iter, W_list)

({('IPCA_reg', 0.1, 0.1): 0.024884449223020466},
 {('IPCA_reg', 0.1, 0.1): 0.0005109145409363247})

In [11]:
cv.cross_val_IPCA(y,x, 0.9, gamma_first, max_iter)

done


({'IPCA': 0.029925434516756555}, {'IPCA': -0.21153091124492174})

In [17]:
data2 = data.copy()
data2 = np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
tk = 1
l = 10
K = kr.K_LR(data2, tk, l)

In [91]:
K

array([[1.        , 0.43468367, 0.66015196, ..., 0.49428864, 0.51621332,
        0.38415098],
       [0.43468367, 1.        , 0.55075558, ..., 0.57294949, 0.44963048,
        0.47853038],
       [0.66015196, 0.55075558, 1.        , ..., 0.68420503, 0.66606611,
        0.48227873],
       ...,
       [0.49428864, 0.57294949, 0.68420503, ..., 1.        , 0.6854606 ,
        0.56743446],
       [0.51621332, 0.44963048, 0.66606611, ..., 0.6854606 , 1.        ,
        0.64552706],
       [0.38415098, 0.47853038, 0.48227873, ..., 0.56743446, 0.64552706,
        1.        ]])

In [54]:
Omega1=np.eye((len(data)-1)*N)
Omega2=np.eye(N)
f_list_kr, v_kr, Q_kr, _, _ = kr.kernel_regression(data, ret, f_list.copy(), lambda1, lambda2, Omega1, Omega2, 10, N, K)

0
1
2
3
4
5
6
7
8
9


In [55]:
print(metrics.total_R_squared_kr(ret, v_kr, Q_kr))

0.14609966865070023


In [18]:
m_hat = 10
L, B = kr.pivoted_chol(K, m_hat)

1
2
3
4
5
6
7
8
9


In [93]:
np.linalg.norm(L@L.T - K)

2046.1921825040288

In [49]:
f_list_kr_LR, v_kr_LR = kr.kernel_regression_LR(data, K, B, ret, f_list.copy(), lambda1, lambda2, Omega2, 10, m_hat, N)

0
1
2
3
4
5
6
7
8
9


In [50]:
print(metrics.total_R_squared_kr_LR(ret, B, K, v_kr_LR, f_list_kr_LR))

0.0017402896001241785


In [13]:
alphas_v = list([1])
cv.cross_val_gaussian(y,x, 0.8, lambda1_v, lambda2_v, alphas_v, N, f_list, Omega2, 10)

0
1
2
3
4
5
6
7
8
9
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done


{('Gaussian', 0.1, 0.1, 1): 0.6073464811374132}

In [21]:
cv.cross_val_gaussian_LR(y, x, 0.8, lambda1_v, lambda2_v, alphas_v, N, f_list, Omega2, 10, m_hat)


1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
39.86480749253068
0
1
2
3
4
5
6
7
8
9
0.001


{('Gaussian', 0.1, 0.1, 1): 0.0}